## Neo4j - CRUD


In this notebook, we will explore Neo4j python low and high level.

Links to documentation:

[neo4j](https://neo4j.com/docs/python-manual/current/)
    
[cypher language (neo4j)](https://neo4j.com/docs/cypher-manual/current/queries/basic/?utm_source=GSearch&utm_medium=PaidSearch&utm_campaign=Evergreen&utm_content=AMS-Search-SEMCE-DSA-None-SEM-SEM-NonABM&utm_term=&utm_adgroup=DSA&gad_source=1&gclid=CjwKCAiAzba9BhBhEiwA7glbaqQWKR6jY5YgSYtbJyZoZZP4kzOuxEXhyOGFnWbSqF-LdfLA1s3r1RoCMUAQAvD_BwE)

[py2neo](https://neo4j-contrib.github.io/py2neo/)

In [1]:
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship,NodeMatcher, RelationshipMatcher
from py2neo.ogm import Model, Property, RelatedTo, RelatedFrom,GraphObject,RelatedObjects

In [2]:
URI = "bolt://192.168.0.17:7687" 
USERNAME = 'neo4j'
PASSWORD = 'changeme'
AUTH = (USERNAME,PASSWORD) 

In [3]:

driver_low_level = GraphDatabase.driver(URI, auth=AUTH)
driver_high_level = Graph(URI, auth=AUTH)

In [ ]:

driver_low_level.verify_connectivity()

## First step - Write

In [ ]:
with driver_low_level.session() as session:
    result = session.run("MATCH (p:Person) DETACH DELETE p")

In [ ]:
query =  "CREATE (p:Person {age: $age, name:$name}) RETURN p"
parameters = {
    "age":42,
    "name":"Ana"
}

with driver_low_level.session() as session:
    result = session.run(query, parameters)

In [ ]:
query =  "MATCH (p:Person) WHERE p.name = $name RETURN p"
parameters = {"name":"Ana"}
with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(f"{result_execute}")

In [ ]:
query =  """
    CREATE (parent:Person {age: $father_age, name:$father_name})
    CREATE (child:Person {name: $child_name, age: $child_age})
    MERGE (parent)-[connect:PARENT_OF {type: $type_parent}]->(child)
    RETURN parent, child, connect
"""
parameters = {
    "father_age":42,
    "father_name":"João",
    "child_age":10,
    "child_name":"Joãozinho",
    "type_parent":"father-son"
}
with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(f"{result_execute}")

In [ ]:
query =  """
    MERGE (parent:Person {age: $mother_age, name:$mother_name})
    MERGE (child:Person {name: $child_name, age: $child_age})
    MERGE (parent)-[connect:PARENT_OF {type: $type_parent}]->(child)
    RETURN parent, child,connect
"""

parameters = {
    "mother_age":42,
    "mother_name":"Maria",
    "child_age":10,
    "child_name":"Joãozinho",
    "type_parent":"mother-son"
}

with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(result_execute)

In [ ]:
query =  """
    MATCH (i:Person)-[:PARENT_OF]-(child:Person {name:$name})
    RETURN i
"""

parameters = {
    "name":"Joãozinho",
}

with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(result_execute)

In [ ]:
with driver_low_level.session() as session:
    result = session.run("MATCH (p:Movie) DETACH DELETE p")

In [ ]:

class Movie(Model):
    title = Property()
    tagline = Property()
    released = Property()


In [ ]:
ace_ventura = Movie(title='ace ventura',tagline='comedia',released=1996)

In [ ]:
driver_high_level.push(ace_ventura)

In [ ]:
Movie.match(driver_high_level).where(title="ace ventura").first()

In [ ]:
Movie.match(driver_high_level).where(title="ace ventura").first().title

In [ ]:
class Artist(Model):
    name = Property()
    born = Property()
    inmovie = RelatedTo("Movie","INMOVIE")

In [ ]:
keanu = Artist(name="Keanu Reeves", born=1964)
matrix = Movie(title="The Matrix", released=1999, tagline="Welcome to the Real World.")

In [ ]:
driver_high_level.push(keanu)
driver_high_level.push(matrix)

In [ ]:
keanu.inmovie.add(matrix,role="main_caracter")

In [ ]:
driver_high_level.push(keanu)

In [ ]:
keanu_return = Artist.match(driver_high_level).where(name="Keanu Reeves").first()

In [ ]:
keanu_return.inmovie.relationship_type

In [ ]:
keanu_return.inmovie.related_class

In [ ]:
keanu_return.inmovie.node

## Second step - Read/Search

In [ ]:
query =  """
    MATCH (i:Person)-[j:PARENT_OF]-(child:Person {name:$name})
    RETURN i,j
"""

parameters = {
    "name":"Joãozinho",
}

with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(f"Len {len(result_execute)}")
    print(result_execute[0])
    print("==============")
    print(result_execute[1])

In [ ]:
query =  """
    MATCH (i:Person)-[j:PARENT_OF]-(child:Person)
    WHERE child.name = $name AND i.name = $iname
    RETURN i,j
"""

parameters = {
    "name":"Joãozinho",
    "iname":"João"
}

with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(f'Len {len(result_execute)}')
    print(result_execute[0])

In [ ]:
query =  """
    MATCH (i:Person)-[j:PARENT_OF]-(child:Person)
    WHERE j.type = 'mother-son'
    RETURN i,j,child
"""

parameters = {
    "name":"Joãozinho",
    "iname":"João"
}

with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(f'Len {len(result_execute)}')
    print(result_execute[0])

In [ ]:
query =  """
    MATCH (i:Person)-[j:PARENT_OF]-(child:Person)
    WHERE ((j.type = 'mother-son') OR (j.type = 'father-son'))
    RETURN i,j,child
"""

parameters = {
    "name":"Joãozinho",
    "iname":"João"
}

with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(f'Len {len(result_execute)}')
    print(result_execute[0])

In [ ]:
Artist.match(driver_high_level).where(name="Keanu Reeves").first()

In [ ]:
node_matcher = NodeMatcher(driver_high_level)
rel_matcher = RelationshipMatcher(driver_high_level)

In [ ]:
keanu = node_matcher.match("Artist", name="Keanu Reeves").first()
movie_rel = rel_matcher.match(r_type="INMOVIE").first()

In [ ]:
movie_rel

In [ ]:
rel_matcher.match(r_type="PARENT_OF",type = 'mother-son').first()

## Third Step - Update

In [ ]:
query =  """
    MATCH (parent:Person)
    WHERE parent.name = $name
    SET parent.age = $age
    SET parent.is_death = False
    RETURN parent
"""
parameters = {
    "age":42,
    "name":"João",
}
with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(f"{result_execute}")

In [ ]:
query =  """
    MATCH (parent:Person)
    WHERE parent.name = $name
    RETURN parent
"""
parameters = {
    "name":"João",
}
with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(f"{result_execute}")

In [ ]:
query =  """
    MATCH (parent:Person)-[]-(i:Person)-[]-(parent2:Person)
    WHERE parent.name = $name AND parent.age = $age
    SET parent2.is_rela_joao = True
    RETURN parent2
"""
parameters = {
    "age":42,
    "name":"João",
}
with driver_low_level.session() as session:
    result = session.run(query, parameters)
    result_execute = [record for record in result]
    print(f"{result_execute}")

## Fourth Step - Delete

## Fifth Step - Extra search

In [5]:
import pandas as pd

In [6]:
df_city = pd.read_csv('../data/city_table.csv')
df_people = pd.read_csv('../data/people_table.csv')
df_family = pd.read_csv('../data/family_table.csv')

In [7]:
def create_constrainx(tx):
    query = """
    CREATE CONSTRAINT unique_city_id IF NOT EXISTS 
    FOR (c:City) REQUIRE c.city_id IS UNIQUE;
    """
    tx.run(query)

def insert_city(tx, row):
    query = """
    MERGE (c:City {city_id: $city_id})
    SET c.name = $name_of_city, c.population = $population_city, c.is_capital = $is_capital
    """
    tx.run(query, 
           city_id=row["city_id"], 
           name_of_city=row["name_of_city"], 
           population_city=row["population_city"], 
           is_capital=row["is_capital"])


def insert_family(tx, row):
    query = """
    MERGE (f:Family {family_id: $family_id})
    SET f.name = $family_name, f.origin_country = $origin_country
    """
    tx.run(query, 
           family_id=row["family_id"], 
           family_name=row["family_name"], 
           origin_country=row["origin_country"])


def insert_person(tx, row):
    query = """
    MERGE (p:Person {people_id: $people_id})
    SET p.name = $name
    
    WITH p
    MATCH (c:City {city_id: $city_id})
    MERGE (p)-[:LIVES_IN]->(c)
    
    WITH p
    MATCH (f:Family {family_id: $family_id})
    MERGE (p)-[:BELONGS_TO]->(f)
    """
    tx.run(query, 
           people_id=row["people_id"], 
           name=row["name"], 
           city_id=row["city_id"], 
           family_id=row["family_id"])


with driver_low_level.session() as session:
    session.write_transaction(create_constrainx)
    print("Add city")
    for _, row in df_city.iterrows():
        session.write_transaction(insert_city, row)
    print("Add family")
    for _, row in df_family.iterrows():
        session.write_transaction(insert_family, row)
    print("Add people")
    for _, row in df_people.iterrows():
        session.write_transaction(insert_person, row)


C:\Users\spguilhermem\AppData\Local\Temp\ipykernel_24608\213221550.py:52: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_constrainx)


Add city


C:\Users\spguilhermem\AppData\Local\Temp\ipykernel_24608\213221550.py:55: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_city, row)


Add family


C:\Users\spguilhermem\AppData\Local\Temp\ipykernel_24608\213221550.py:58: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_family, row)


Add people


C:\Users\spguilhermem\AppData\Local\Temp\ipykernel_24608\213221550.py:61: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(insert_person, row)


In [ ]:
def create_project(tx):
    query = """
    CALL gds.graph.project(
        'communityGraph',  // Name of the in-memory graph
        ['Person', 'City', 'Family'],  // Nodes to include
        {LIVES_IN: {}, BELONGS_TO: {}}  // Relationships to include
    );
    """
    tx.run(query)


def louvain(tx):
    query = """
    CALL gds.louvain.write(
        'communityGraph',  // Use the projected graph
        {writeProperty: 'communityId'}
    );
    """
    tx.run(query)